In [19]:
using Plots
using DataFrames

In [ ]:
MX = 201
I1 = 95
I2 = 105
MY = 401
J1 = 95
J2 = 105
MAXITP = 10000
OMEGAP = 1.0 
ERRORP = 1e-4
Re = 70

70

In [3]:
function BoundaryConditionforP(p::Array{Float64, 2})
    for i in 1:MY
        p[1, i] = 0.0
        p[MX, i] = 0.0
    end
    
    for i in 1:MX
        p[i, 1] = 0.0
        p[i, MY] = 0.0
    end
    
    p[I1, J1] = p[I1-1, J1-1]
    p[I1, J2] = p[I1-1, J2+1]
    p[I2, J1] = p[I2+1, J1-1]
    p[I2, J2] = p[I2+1, J2+1]
    
    for j in J1+1:J2-1
        p[I1, j] = p[I1-1, j]
        p[I2, j] = p[I2+1, j]
    end
    
    for i in I1+1:I2-1
        p[i, J1] = p[i, J1-1]
        p[i, J2] = p[i, J2+1]
    end
end

BoundaryConditionforP (generic function with 1 method)

In [ ]:
function BoundaryConditionforV(u::Array{Float64, 2}, v::Array{Float64, 2})
    for j in 1:MY
        u[1, j] = 1.0
        v[1, j] = 0.0
        u[MX, j] = 2.0 * u[MX-1, j] - u[MX-2, j]
        v[MX, j] = 2.0 * v[MX-1, j] - v[MX-2, j]
    end
    
    for i in 1:MX
        u[i, 1] = 2.0 * u[i, 2] - u[i, 3]
        v[i, 1] = 2.0 * v[i, 2] - v[i, 3]
        u[i, MY] = 2.0 * u[i, MY-1] - u[i, MY-2]
        v[i, MY] = 2.0 * v[i, MY-1] - v[i, MY-2]
    end
    
    for i in I1:I2
        for j in J1:J2
            u[i, j] = 0.0
            v[i, j] = 0.0
        end
    end
end

BoundaryConditionforV (generic function with 1 method)

In [ ]:
function PoissonEquation(u::Array{Float64, 2}, v::Array{Float64, 2}, p::Array{Float64, 2},
                         dx::Float64, dy::Float64, dt::Float64)
    rhs = zeros(MX, MY)
    
    for i in 2:MX-1
        for j in 2:MY-1
            if (I1 < i < I2) && (J1 < j < J2) 
            else
                ux = (u[i+1, j] - u[i-1, j]) / (2.0*dx)
                uy = (u[i, j+1] - u[i, j-1]) / (2.0*dy)
                vx = (v[i+1, j] - v[i-1, j]) / (2.0*dx)
                vy = (v[i, j+1] - v[i, j-1]) / (2.0*dy)
                rhs[i, j] = (ux + vy) / dt - (ux^2 + 2.0 * uy * vx + vy^2)
            end
        end
    end
    
    @inbounds for iteration in 1:MAXITP
        res = 0.0
        for i in 2:MX-1
            for j in 2:MY-1
                if (I1 <= i <= I2) && (J1 <= j <= J2)
                else
                    dp = (p[i+1, j] + p[i-1, j]) / dx^2 + (p[i, j+1] + p[i, j-1]) / dy^2 -rhs[i, j]
                    dp = dp / (2.0/dx^2 + 2.0/dy^2) - p[i, j]
                    res += dp^2
                    p[i, j] += OMEGAP * dp
                end
            end
        end
        BoundaryConditionforP(p)
        res = sqrt(res / (MX * MY))
        if res < ERRORP
            resp = res
            itrp = iteration
            break
        end
    end
end

PoissonEquation (generic function with 1 method)

In [ ]:
function VelocityEquation(u::Array{Float64, 2}, v::Array{Float64, 2}, p::Array{Float64, 2},
                          dx::Float64, dy::Float64, dt::Float64)
    urhs = zeros(MX, MY)
    vrhs = zeros(MX, MY)
    for i in 2:MX-1
        for j in 2:MY-1
            if (I1 < i < I2) && (J1 < j < J2)
            elseif (i == I1 || i== I2) && (J1 < j < J2)
                urhs[i, j] = 0.0
                vrhs[i, j] = -(p[i, j+1] - p[i, j-1]) / (2.0*dy)
            elseif (I1 < i < I2) && (j == J1 || j == J2)
                urhs[i, j] = -(p[i+1, j] - p[i-1, j]) / (2.0*dx)
                vrhs[i, j] = 0.0
            elseif (i == I1 || i == I2) && (j == J1 || j == J2)
                urhs[i, j] = 0.0
                vrhs[i, j] = 0.0
            else
                urhs[i, j] = -(p[i+1, j] - p[i-1, j]) / (2.0*dx)
                vrhs[i, j] = -(p[i, j+1] - p[i, j-1]) / (2.0*dy)
            end
        end
    end
    
    for i in 2:MX-1
        for j in 2:MY-1
            if (I1 < i <I2) && (J1 < j < J2)
            else
                urhs[i, j] += (u[i+1, j] - 2.0*u[i, j] + u[i-1, j]) / (Re*dx^2) + (u[i, j+1] - 2.0*u[i, j] + u[i, j-1]) / (Re*dy^2)
                vrhs[i, j] += (v[i+1, j] - 2.0*v[i, j] + v[i-1, j]) / (Re*dx^2) + (v[i, j+1] - 2.0*v[i, j] + v[i, j-1]) / (Re*dy^2)
            end
        end
    end
    
    for j in J1+1:J2-1
        u[I1+1, j] = 2.0*u[I1, j] - u[I1-1, j]
        u[I2-1, j] = 2.0*u[I2, j] - u[I2+1, j]
        v[I1+1, j] = 2.0*v[I1, j] - v[I1-1, j]
        v[I2-1, j] = 2.0*v[I2, j] - v[I2+1, j]
    end
    
    for i in 2:MX-1
        for j in 2:MY-1
            if (I1 < i < I2) && (J1 < j < J2)
            elseif i == 2
                urhs[i, j] -= u[i, j] * (-u[i+2, j] + 8.0*(u[i+1, j] - u[i-1, j]) + u[i-1, j]) / (12.0*dx) -
                abs(u[i, j]) * (u[i+2, j] - 4.0*u[i+1, j] + 6.0*u[i, j] - 4.0*u[i-1, j] + u[i-1, j]) / (4.0*dx)
                vrhs[i, j] -= u[i, j] * (-v[i+2, j] + 8.0*(v[i+1, j] - v[i-1, j]) + v[i-1, j]) / (12.0*dx) -
                abs(u[i, j]) * (v[i+2, j] - 4.0*v[i+1, j] + 6.0*v[i, j] - 4.0*v[i-1, j] + v[i-1, j]) / (4.0*dx)
            elseif i == MX-1
                urhs[i, j] -= u[i, j] * (-2.0 * u[i+1, j] + u[i, j] + 8 * (u[i+1, j] - u[i-1, j]) + u[i-2, j]) / (12.0*dx) + 
                abs(u[i, j]) * (2.0 * u[i+1, j] - u[i, j] - 4.0 * u[i+1, j] + 6.0 * u[i, j] - 4.0 * u[i-1, j] + u[i-2, j]) / (4.0*dx)
                vrhs[i, j] -= u[i, j] * (-2.0 * v[i+1, j] + v[i, j] + 8 * (v[i+1, j] - v[i-1, j]) + v[i-2, j]) / (12.0*dx) + 
                abs(u[i, j]) * (2.0 * v[i+1, j] - v[i, j] - 4.0 * v[i+1, j] + 6.0 * v[i, j] - 4.0 * v[i-1, j] + v[i-2, j]) / (4.0*dx)
            else
                urhs[i, j] -= u[i, j] * (-u[i+2, j] + 8.0 * (u[i+1, j] - u[i-1, j]) + u[i-2, j]) / (12.0*dx) + 
                abs(u[i, j]) * (u[i+2, j] - 4.0 * u[i+1, j] + 6.0 * u[i, j] - 4.0 * u[i-1, j] + u[i-2, j]) / (4.0*dx)
                vrhs[i, j] -= u[i, j] * (-v[i+2, j] + 8.0 * (v[i+1, j] - v[i-1, j]) + v[i-2, j]) / (12.0*dx) + 
                abs(u[i, j]) * (v[i+2, j] - 4.0 * v[i+1, j] + 6.0 * v[i, j] - 4.0 * v[i-1, j] + v[i-2, j]) / (4.0*dx)
            end
        end
    end
    
    for i in I1+1:I2-1
        u[i, J1 + 1] = 2.0 * u[i, J1] - u[i, J1 - 1]
        u[i, J2 - 1] = 2.0 * u[i, J2] - u[i, J2 + 1]
        v[i, J1 + 1] = 2.0 * v[i, J1] - v[i, J1 - 1]
        v[i, J2 - 1] = 2.0 * v[i, J2] - v[i, J2 + 1]
    end
    
    for i in 2:MX-1
        for j in 2:MY-1
            if (I1 < i < I2) && (J1 < j < J2)
            elseif j == 2
                urhs[i, j] -= v[i, j] * (-u[i, j+2] + 8.0 * (u[i, j+1] - u[i, j-1]) + 2.0 * u[i, j-1] - u[i, j]) / (12.0*dy) + 
                abs(v[i, j]) * (u[i, j+2] - 4.0 * u[i, j+1] + 6.0 * u[i, j] - 4.0 * u[i, j-1] + 2.0 * u[i, j-1] - u[i, j]) / (4.0*dy)
                vrhs[i, j] -= v[i, j] * (-v[i, j+2] + 8.0 * (v[i, j+1] - v[i, j-1]) + 2.0 * v[i, j-1] - v[i, j]) / (12.0*dy) + 
                abs(v[i, j]) * (v[i, j+2] - 4.0 * v[i, j+1] + 6.0 * v[i, j] - 4.0 * v[i, j-1] + 2.0 * v[i, j-1] - v[i, j]) / (4.0*dy)
            elseif j == MY-1
                urhs[i, j] -= v[i, j] * (-2.0 * u[i, j+1] + u[i, j] + 8.0 * (u[i, j+1] - u[i, j-1]) + u[i, j-2]) / (12.0*dy) + 
                abs(v[i, j]) * (2.0 * u[i, j+1] - u[i, j] - 4.0 * u[i, j+1] + 6.0 * u[i, j] - 4.0 * u[i, j-1] + u[i, j-2]) / (4.0*dy)
                vrhs[i, j] -= v[i, j] * (-2.0 * v[i, j+1] + v[i, j] + 8.0 * (v[i, j+1] - v[i, j-1]) + v[i, j-2]) / (12.0*dy) + 
                abs(v[i, j]) * (2.0 * v[i, j+1] - v[i, j] - 4.0 * v[i, j+1] + 6.0 * v[i, j] - 4.0 * v[i, j-1] + v[i, j-2]) / (4.0*dy)
            else
                urhs[i, j] -= v[i, j] * (-u[i, j+2] + 8.0 * (u[i, j+1] - u[i, j-1]) + u[i, j-2]) / (12.0*dy) + 
                abs(v[i, j]) * (u[i, j+2] - 4.0 * u[i, j+1] + 6.0 * u[i, j] - 4.0 * u[i, j-1] + u[i, j-2]) / (4.0*dy)
                vrhs[i, j] -= v[i, j] * (-v[i, j+2] + 8.0 * (v[i, j+1] - v[i, j-1]) + v[i, j-2]) / (12.0*dy) + 
                abs(v[i, j]) * (v[i, j+2] - 4.0 * v[i, j+1] + 6.0 * v[i, j] - 4.0 * v[i, j-1] + v[i, j-2]) / (4.0*dy)
            end
        end
    end
    
    for i in 2:MX-1
        for j in 2:MY-1
            if (I1 < i < I2) && (J1 < j <J2)
            else
                u[i, j] += dt * urhs[i, j]
                v[i, j] += dt * vrhs[i, j]
            end
        end
    end
end

VelocityEquation (generic function with 1 method)

In [20]:
function main()
    CFL = 0.2
    nlast = 5000
    nlp = 10
    
    dx = 1.0 / (I2 - I1)
    dy = 1.0 / (J2 - J1)
    icent = (I2 + I1) / 2
    jcent = (J2 + J1) / 2
    x = zeros(MX, MY)
    y = zeros(MX, MY)
    
    for i in 1:MX
        for j in 1:MY
            x[i, j] = dx * (i - icent)
            y[i, j] = dy * (j - jcent)
        end
    end
    
    dt = CFL * min(dx, dy)
    nbegin = 0
    time = 0.0
    u = ones(MX, MY)
    v = zeros(MX, MY)
    p = zeros(MX, MY)
    
    BoundaryConditionforP(p)
    BoundaryConditionforV(u, v)
    
    fp = open("logp.csv", "w")
    fu = open("logu.csv", "w")
    fv = open("logv.csv", "w")
    
    write(fp, DataFrame(p))
    write(fu, DataFrame(u))
    write(fv, DataFrame(v))
    
    close(fp)
    close(fu)
    close(fv)
    
    for i in 1:nlast
        println(i)
        nstep = i + nbegin
        time += dt
        PoissonEquation(u, v, p, dx, dy, dt)
        BoundaryConditionforP(p)
        VelocityEquation(u, v, p, dx, dy, dt)
        BoundaryConditionforV(u, v)
        if i % 10 == 0
            fp = open("logp.csv", "a")
            fu = open("logu.csv", "a")
            fv = open("logv.csv", "a")
    
            write(fp, DataFrame(p))
            write(fu, DataFrame(u))
            write(fv, DataFrame(v))
            
            close(fp)
            close(fu)
            close(fv)
        end
    end
end

main (generic function with 1 method)

In [21]:
using CSV
@time main()

LoadError: [91mMethodError: no method matching write(::IOStream, ::DataFrames.DataFrame)[0m
Closest candidates are:
  write(::IO, ::Any) at io.jl:284
  write(::IO, ::Any...) at io.jl:286
  write(::IOStream, [91m::Char[39m) at iostream.jl:214
  ...[39m

In [22]:
p = CSV.read("../csvfiles/logp.csv")

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100,x101,x102,x103,x104,x105,x106,x107,x108,x109,x110,x111,x112,x113,x114,x115,x116,x117,x118,x119,x120,x121,x122,x123,x124,x125,x126,x127,x128,x129,x130,x131,x132,x133,x134,x135,x136,x137,x138,x139,x140,x141,x142,x143,x144,x145,x146,x147,x148,x149,x150,x151,x152,x153,x154,x155,x156,x157,x158,x159,x160,x161,x162,x163,x164,x165,x166,x167,x168,x169,x170,x171,x172,x173,x174,x175,x176,x177,x178,x179,x180,x181,x182,x183,x184,x185,x186,x187,x188,x189,x190,x191,x192,x193,x194,x195,x196,x197,x198,x199,x200,x201
1,Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(0.0),Nullable{Float64}(

In [23]:
typeof(p)

DataFrames.DataFrame

In [25]:
p[205, 100]

Nullable{Float64}(0.0)

In [26]:
MX = 401
MY = 201

201

In [27]:
dx = 0.1
dy = 0.1
x = (1:MX) * dx
y = (1:MY) * dy

0.1:0.1:20.1

In [30]:
p1 = contour(x, y, p[1:MX, 1:MY], fill=true)
plot(p1)

LoadError: [91mNo user recipe defined for DataFrames.DataFrame[39m